#Credit Scoring

#Preparação

###Bibliotecas usadas

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.metrics import roc_auc_score
from scipy.stats import ks_2samp


###Funções KS e GINI

In [ ]:
def calcular_ks(y_true, y_pred):
    """
    Calcula a estatística KS (Kolmogorov-Smirnov).

    Args:
        y_true: Array numpy ou lista contendo os valores reais (0 ou 1).
        y_pred: Array numpy ou lista contendo as probabilidades previstas (entre 0 e 1).

    Returns:
        O valor da estatística KS.
    """
    data = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
    data_bom = data[data['y_true'] == 0]['y_pred']
    data_mau = data[data['y_true'] == 1]['y_pred']
    ks = ks_2samp(data_bom, data_mau).statistic
    return ks

In [ ]:
def calcular_gini(y_true, y_pred):
    """
    Calcula o GINI a partir das previsões e valores reais.

    Args:
        y_true: Array numpy ou lista contendo os valores reais (0 ou 1).
        y_pred: Array numpy ou lista contendo as probabilidades previstas (entre 0 e 1).

    Returns:
        O valor do GINI.
    """
    auc = roc_auc_score(y_true, y_pred)
    gini = 2 * auc - 1
    return gini

###Função para reencher valores ausentes (NaN) em uma coluna do DataFrame.

In [ ]:
def fill_missing_values(df, column, filler_values):
    """
    A função fill_missing_values preenche valores ausentes (NaN)
    em uma coluna de um DataFrame com uma lista de valores de preenchimento,
    distribuindo esses valores uniformemente até preencher todos os NaNs
    ou esgotar a lista de valores de preenchimento,
    e imprime o número de NaNs restantes após o preenchimento.

    Args:
        df (pandas.DataFrame): O DataFrame a ser modificado.
        column (str): O nome da coluna a ser preenchida.
        filler_values (list): Uma lista de valores para preencher os NaNs.
    """
    nan_count = df[column].isnull().sum()

    if nan_count > 0:
        for i, filler in enumerate(filler_values):
            limit = int(nan_count // len(filler_values)) if i < len(filler_values) - 1 else None
            df[column].fillna(filler, limit=limit, inplace=True)

    print(f"Valores NaN restantes na coluna '{column}': {df[column].isnull().sum()}")

###Importação dos dados

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DATA_VIKING/credit_risk.csv')

In [ ]:
df.head()

###Tratamento dos dados

In [ ]:
df.info()

In [ ]:
# Tratando valores nulos (NaN)
# Substituindo valores NaN na coluna 'saving_accounts', pela variável 'little'
df['saving_accounts'].fillna('little', inplace=True)

# Substituindo valores NaN na coluna 'checking_account', dividindo entre 'little' e 'moderate'
fill_missing_values(df, 'checking_account', ['little', 'moderate'])

In [ ]:
# Substituindo na coluna 'risk'...{'good': 1, 'bad': 0}
df['risk'] = df['risk'].map({'good': 1, 'bad': 0})

# Substituindo na coluna 'sex'...{'male': 1, 'female': 0}
df['sex'] = df['sex'].map({'male': 1, 'female': 0})

In [ ]:
# Criando as colunas 'month' e 'year', recebendo as respectivas informações da coluna 'reference'
df['month'] = df['reference'].str.split('-').str[1]
df['year'] = df['reference'].str.split('-').str[0]

#Transformando as colunas separadas ('month' e 'year') para o tipo inteiro

df['month'] = df['month'].astype(int)
df['year'] = df['year'].astype(int)

In [ ]:
# Removendo a colunas do DataFrame.
df = df.drop(columns=['Unnamed: 0', 'cpf', 'income', 'reference'])

In [ ]:
categorical_columns = df.select_dtypes(include=['object']).columns
categorical_columns

In [ ]:
for columns in categorical_columns:
  df = pd.get_dummies(df, columns=[columns], drop_first=True, dtype=int)

In [ ]:
df.head()

###Separando coluna Target do resto do DataFrame

In [ ]:
df_x = df.drop(columns=['risk'])
df_y = df['risk']

#Modelo I

In [ ]:
# Dividindo em treino e teste
X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)

# Criando e treinando o modelo
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
# Avaliação do modelo:
# Fazendo previsões no conjunto de teste
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)  # Probabilidades das previsões
y_pred_proba_1 = model.predict_proba(X_test)[:, 1]  # Probabilidades da classe 1

In [ ]:
# Métricas básicas
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))
print("\nMatriz de Confusão:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
# Cálculo das Métricas KS e GINI
ks = calcular_ks(y_test, y_pred_proba_1)
gini = calcular_gini(y_test, y_pred_proba_1)

In [ ]:
print(ks)
print(gini)

In [ ]:
# Calculando a curva ROC
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba[:, 1])
roc_auc = auc(fpr, tpr)

# Plotando a curva
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC')
plt.show()

In [ ]:
# Visualizando os coeficientes
coef_df = pd.DataFrame({
    'Feature': df_x.columns,
    'Coefficient': model.coef_[0]
})
coef_df = coef_df.sort_values('Coefficient', ascending=False)
print("Coeficientes do modelo:")
print(coef_df)

In [ ]:
# Realizando validação cruzada
cv_scores = cross_val_score(model, df_x, df_y, cv=5)
print("Scores da validação cruzada:", cv_scores)
print("Média da validação cruzada:", cv_scores.mean())

In [ ]:
# Definindo parâmetros para teste
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear', 'saga']
}

# Realizando busca em grade
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Melhores parâmetros:", grid_search.best_params_)
print("Melhor score:", grid_search.best_score_)

###Tratamento de desbalanceamento

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [ ]:
# Criar pipeline de balanceamento
balancing_pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('undersampling', RandomUnderSampler(random_state=42))
])

# Aplicar balanceamento
X_train_balanced, y_train_balanced = balancing_pipeline.fit_resample(X_train, y_train)

###Feature engineering

In [ ]:
df.info()

In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# Criar novas features
scaler = StandardScaler()
poly = PolynomialFeatures(degree=2, include_bias=False)

In [ ]:
# Pipeline com feature engineering
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Assumindo que algumas colunas são numéricas e outras categóricas
numeric_features = numeric_features = df.columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        (StandardScaler(), numeric_features),
        ])

###Otimização de hiperparametros

In [ ]:
param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'class_weight': ['balanced', None],
    'solver': ['liblinear', 'saga'],
    'penalty': ['l1', 'l2']
}

In [ ]:
# Criar pipeline completo
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# Grid Search com validação cruzada
grid_search = GridSearchCV(
    model_pipeline,
    param_grid,
    cv=5,
    scoring=['accuracy', 'f1', 'roc_auc'],
    refit='roc_auc'
)

###Validação com K-fold

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

###Métricas para Credit Scoring

In [ ]:
def evaluate_credit_model(y_true, y_pred, y_pred_proba):
    ks = calcular_ks(y_true, y_pred_proba)
    gini = calcular_gini(y_true, y_pred_proba)

    print(f"KS Score: {ks:.4f}")
    print(f"Gini Index: {gini:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))

    # Plotar curva ROC
    plot_roc_curve(y_true, y_pred_proba)

#Modelo II

In [ ]:
# Usar class_weight='balanced'
model = LogisticRegression(class_weight='balanced')

In [ ]:
# Dividindo em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_train_balanced, y_train_balanced, test_size=0.2, random_state=42)

# Criando e treinando o modelo
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
# Avaliação do modelo:
# Fazendo previsões no conjunto de teste
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilidades da classe 1

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Métricas básicas
print("accuracy_score:", accuracy_score(y_test, y_pred))
print("precision_score: ", precision_score(y_test, y_pred))
print("recall_score: ", recall_score(y_test, y_pred))
print("f1_score: ", f1_score(y_test, y_pred))
print("roc_auc_score: ", roc_auc_score(y_test, y_pred_proba))

In [ ]:
print("confusion_matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
# Cálculo das Métricas KS e GINI
ks = calcular_ks(y_test, y_pred_proba)
gini = calcular_gini(y_test, y_pred_proba)

print(ks)
print(gini)